# File Structure

A SoS script usually starts with lines

```python
#!/usr/bin/env sos-runner
#fileformat=SOS1.0
```

The first line allows the script to be executed by command `sos-runner` if it is executed as an executable script. The second line tells SoS the format of the script. The `#fileformat` line does not have to be the first or second line but should be in the first comment block. SOS format 1.0 is assumed if no format line is present.

## Global definitions

Python functions, classes, variables can be defined or imported (using Python `import` statement) before any SoS step is defined. These definitions usually contains variables such as version and date of the script, paths to various resources, and utility functions that will be used by later steps. **These definitions are visible to all steps of workflows and are assumed to be readonly**.

In addition to user-defined global variables, SoS defines the following variables before any variables are defined

* **`SOS_VERSION`**: version of SoS command.
* **`CONFIG`**: A dictionary of configurations specified by the global sos configuration file (`~/.sos/config.yaml`), local configuration file (`~/.sos/config.yaml`) and command line option `-c config_file`. 
* **`run_mode`**: A string indicating the run mode in which the script is executed, which can be `inspect`, `prepare` or `run`. Because the script and therefore all Python statements in the script will be executed in all three modes, this variable allows the execution of some statement only in selected modes.

## `%include` preprocessor

SoS allows you to include variables and steps of another script into the current script using preprocessor `%include`. The `%include` statement should be used before any other SoS statements and follows the same syntax as the python `import` keyword. For example, the following statements are allowed

```
%include alignment
%include alignment as ag
%include alignment, calling
%include alignment as ag, calling as ca

%from alignment include *
%from alignment include var1, workflow1
%from alignment include workflow1 as wf
```

Similar to python `import` statement, variables and workflows included using the `%include ... as ...` syntax can be accessed using `module.name` (e.g. `alignment.var1`, `ag.var1` (for `include alignment as ag`)), whereas variables and workflows included using the `from ... include ...` syntax are available directly (e.g. `var1`, `workflow1`, and `wf` (for `workflow1 as wf`).

For example

```
#!/usr/bin/env sos-runner
#fileformat=SOS1.0

%include alignment
%include call
[10]
sos_run('alignment.default + call.default')
```

would execute two workflows defined in `alignment.sos` and `call.sos`. The files should be in SoS search path to be included.

## `%if .. %elif ..%else .. %endif`

SoS allows you to conditionally include part of the script using macos `%if`, `%elif`, `%else`, and `%endif`. The condition in `%if cond` and `elif cond` should be valid python expression. The condition should not expand multiple lines and should not have trailing `:`. The conditions are evaluated at parsing time so no SoS variables are allowed. For example, you can write

```
%if sys.platform == 'darwin'

[step]
Mac OSX step

%else

[step]
Linux step

%endif
```